# Time Series

## 1 - Data Prep

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import glob
import numpy as np
%matplotlib inline

from datetime import datetime
root = "../"

In [2]:
population = pd.read_csv(root + 'CSV/Population/population_08_18.csv', index_col=[0])
for col in population:
    if col != "MUNCOD":
        year = "20" + col.split("_")[1]
        population = population.rename(columns={col: year})
population = population.astype(int)
population.head()

,2018,MUNCOD,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008
0,23167,110001,25437,25506,25578,25652,25728,24069,24228,24422,24354,24577
1,106168,110002,107345,105896,104401,102860,101269,92747,91570,90354,85541,84581
2,5438,110003,6224,6289,6355,6424,6495,6132,6221,6309,6695,6777
3,84813,110004,88507,87877,87226,86556,85863,79330,78959,78601,78675,78263
4,16444,110005,17934,17959,17986,18013,18041,16852,16939,17030,16622,16784


In [3]:
path = root + 'CSV/TabNet/Internacoes/'
suicide_path = root + 'CSV/TabNet/Suicides/'
df_final = pd.DataFrame()
list_of_diseases = ['TRANSTORNOS_DE_HUMOR_[AFETIVOS]',
 'TRANST_MENTAIS_E_COMPORTAMENTAIS_DEV_USO_DE_ÁLCOOL',
 'NEOPLASIA_MALIGNA_DO_CÓLON',
 'NEOPLASIA_MALIGNA_DE_TRAQUÉIA_BRÔNQUIOS_E_PULMÕES',
 'TRANST_MENTAIS_COMPORT_DEV_USO_OUTR_SUBST_PSICOAT',
 'BRONQUITE_ENFISEMA_E_OUTR_DOENÇ_PULM_OBSTR_CRÔNIC',
 'NEOPL_MALIG_JUNÇÃO_RETOSSIGM_RETO_ÂNUS_CANAL_ANAL',
 'OUTRAS_NEOPLASIAS_MALIGNAS_DA_PELE',
 'TRANSTORNOS_DE_CONDUÇÃO_E_ARRITMIAS_CARDÍACAS',
 'FLEBITE_TROMBOFLEBITE_EMBOLIA_E_TROMBOSE_VENOSA',
 'OUTRAS_DOENÇAS_ISQUÊMICAS_DO_CORAÇÃO',
 'TRANSTORNOS_DOS_NERVOS_RAÍZES_E_PLEXOS_NERVOSOS',
 'NEOPL_MALIG_OUTR_LOCALIZ_MAL_DEF_SECUN_E_NÃO_ESPEC',
 'DOENÇAS_DO_APÊNDICE',
 'COLELITÍASE_E_COLECISTITE',
 'NEOPLASIA_MALIGNA_DO_ESÔFAGO',
 'OUTROS_TRANSTRONOS_ARTICULARES',
 'UROLITÍASE',
 'PANCREATITE_AGUDA_E_OUTRAS_DOENÇAS_DO_PÂNCREAS',
 'TRANSTORNOS_DA_DENSIDADE_E_DA_ESTRUTURA_ÓSSEAS']
month_dic = {
    "Jan": "January",
    "Fev": "February",
    "Mar": "March",
    "Abr": "April",
    "Mai": "May",
    "Jun": "June",
    "Jul": "July",
    "Ago": "August",
    "Set": "September",
    "Out": "October",
    "Nov": "November",
    "Dez": "December"
}

def prepare_df(df):
    df["MUNCOD"] = [x.split(" ")[0] for x in df["Município"]]
    df = df[df["MUNCOD"] != "Total"]
    df = df.filter(regex="^((?!2007|2019|2020).)*$")
    df = df.drop(columns=["Município"])
    df = df.replace("-",0)
    df = df.astype(int)
    return df
for disease in list_of_diseases:
    df_disease = pd.read_csv(path + disease + ".csv", sep=";")
    df_disease = prepare_df(df_disease)
    df_disease = df_disease.drop(columns=["Total"])
    years = [str(x) for x in range(2008,2019)]
    for year in years:
        suicide_df = pd.read_csv(suicide_path + "suicide_"+ year + ".csv", sep=";")
        suicide_df = prepare_df(suicide_df)
        suicide_df = suicide_df[["Total", "MUNCOD"]]
        count = df_disease.filter(regex=year, axis=1).sum(axis=1)
        df_disease = df_disease.filter(regex="^((?!" + year+ ").)*$")
        df_disease["QTD_" + year] = count
        df_disease = pd.merge(suicide_df, df_disease, on="MUNCOD")
        df_pop = population[["MUNCOD", year]]
        df_disease = pd.merge(df_disease, df_pop, on="MUNCOD")
        df_disease["QTD_" + year] = df_disease["QTD_" + year] / df_disease[year] * 100000
        df_disease["SUICIDE_" + year] = df_disease["Total"] / df_disease[year] * 100000
        df_disease = df_disease.drop(columns=[year, "Total"])
    years.pop()
    years.pop(0)
    df_all_years = pd.DataFrame()
    for year in years:
        next_year = str(int(year) + 1)
        last_year = str(int(year) - 1)
        df_med = df_disease[["MUNCOD", "QTD_" + year, "QTD_" + last_year, "SUICIDE_" + year, "SUICIDE_" + next_year]].copy()
        df_med["YEAR"] = year
        df_med.columns = ["MUNCOD", disease,"PREVIOUS_" + disease, "PREVIOUS", "TARGET", "YEAR"]
        df_all_years = pd.concat([df_all_years,df_med])
    if df_final.empty:
        df_final = df_all_years
    else:
        df_all_years = df_all_years[["MUNCOD", "PREVIOUS_" + disease,disease, "YEAR"]]
        df_final = pd.merge(df_final, df_all_years, on=["MUNCOD", "YEAR"])
df_final.head()

,MUNCOD,TRANSTORNOS_DE_HUMOR_[AFETIVOS],PREVIOUS_TRANSTORNOS_DE_HUMOR_[AFETIVOS],PREVIOUS,TARGET,YEAR,PREVIOUS_TRANST_MENTAIS_E_COMPORTAMENTAIS_DEV_USO_DE_ÁLCOOL,TRANST_MENTAIS_E_COMPORTAMENTAIS_DEV_USO_DE_ÁLCOOL,PREVIOUS_NEOPLASIA_MALIGNA_DO_CÓLON,NEOPLASIA_MALIGNA_DO_CÓLON,...,PREVIOUS_NEOPLASIA_MALIGNA_DO_ESÔFAGO,NEOPLASIA_MALIGNA_DO_ESÔFAGO,PREVIOUS_OUTROS_TRANSTRONOS_ARTICULARES,OUTROS_TRANSTRONOS_ARTICULARES,PREVIOUS_UROLITÍASE,UROLITÍASE,PREVIOUS_PANCREATITE_AGUDA_E_OUTRAS_DOENÇAS_DO_PÂNCREAS,PANCREATITE_AGUDA_E_OUTRAS_DOENÇAS_DO_PÂNCREAS,PREVIOUS_TRANSTORNOS_DA_DENSIDADE_E_DA_ESTRUTURA_ÓSSEAS,TRANSTORNOS_DA_DENSIDADE_E_DA_ESTRUTURA_ÓSSEAS
0,110001,0.000000,8.137690,8.212203,8.189337,2009,0.000000,4.106102,0.000000,0.000000,...,0.000000,0.000000,4.068845,12.318305,142.409570,119.076948,0.000000,4.106102,4.068845,0.000000
1,110002,4.676120,1.182299,2.338060,4.427031,2009,1.182299,2.338060,5.911493,2.338060,...,3.546896,9.352240,7.093792,11.690301,15.369882,43.254112,5.911493,3.507090,7.093792,2.338060
2,110004,1.271052,1.277743,7.626311,2.544497,2009,0.000000,0.000000,3.833229,8.897363,...,2.555486,6.355259,7.666458,8.897363,63.887150,25.421036,3.833229,5.084207,6.388715,3.813155
3,110080,11.397960,0.000000,5.698980,10.110201,2009,0.000000,0.000000,0.000000,0.000000,...,0.000000,11.397960,0.000000,5.698980,11.663848,11.397960,11.663848,5.698980,11.663848,0.000000
4,110010,4.906531,4.933277,7.359796,2.401191,2009,2.466639,0.000000,0.000000,0.000000,...,0.000000,0.000000,9.866555,22.079388,12.333194,22.079388,4.933277,2.453265,0.000000,0.000000


In [4]:
df = df_final.drop(columns=["MUNCOD"])
df = df.astype(float)
df.head()

,TRANSTORNOS_DE_HUMOR_[AFETIVOS],PREVIOUS_TRANSTORNOS_DE_HUMOR_[AFETIVOS],PREVIOUS,TARGET,YEAR,PREVIOUS_TRANST_MENTAIS_E_COMPORTAMENTAIS_DEV_USO_DE_ÁLCOOL,TRANST_MENTAIS_E_COMPORTAMENTAIS_DEV_USO_DE_ÁLCOOL,PREVIOUS_NEOPLASIA_MALIGNA_DO_CÓLON,NEOPLASIA_MALIGNA_DO_CÓLON,PREVIOUS_NEOPLASIA_MALIGNA_DE_TRAQUÉIA_BRÔNQUIOS_E_PULMÕES,...,PREVIOUS_NEOPLASIA_MALIGNA_DO_ESÔFAGO,NEOPLASIA_MALIGNA_DO_ESÔFAGO,PREVIOUS_OUTROS_TRANSTRONOS_ARTICULARES,OUTROS_TRANSTRONOS_ARTICULARES,PREVIOUS_UROLITÍASE,UROLITÍASE,PREVIOUS_PANCREATITE_AGUDA_E_OUTRAS_DOENÇAS_DO_PÂNCREAS,PANCREATITE_AGUDA_E_OUTRAS_DOENÇAS_DO_PÂNCREAS,PREVIOUS_TRANSTORNOS_DA_DENSIDADE_E_DA_ESTRUTURA_ÓSSEAS,TRANSTORNOS_DA_DENSIDADE_E_DA_ESTRUTURA_ÓSSEAS
0,0.000000,8.137690,8.212203,8.189337,2009.0,0.000000,4.106102,0.000000,0.000000,0.000000,...,0.000000,0.000000,4.068845,12.318305,142.409570,119.076948,0.000000,4.106102,4.068845,0.000000
1,4.676120,1.182299,2.338060,4.427031,2009.0,1.182299,2.338060,5.911493,2.338060,0.000000,...,3.546896,9.352240,7.093792,11.690301,15.369882,43.254112,5.911493,3.507090,7.093792,2.338060
2,1.271052,1.277743,7.626311,2.544497,2009.0,0.000000,0.000000,3.833229,8.897363,1.277743,...,2.555486,6.355259,7.666458,8.897363,63.887150,25.421036,3.833229,5.084207,6.388715,3.813155
3,11.397960,0.000000,5.698980,10.110201,2009.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,11.397960,0.000000,5.698980,11.663848,11.397960,11.663848,5.698980,11.663848,0.000000
4,4.906531,4.933277,7.359796,2.401191,2009.0,2.466639,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,9.866555,22.079388,12.333194,22.079388,4.933277,2.453265,0.000000,0.000000


In [ ]:
# df = df[df["YEAR"] == 2017]

In [5]:
from sklearn.model_selection import train_test_split
X = df.drop(columns=["TARGET", "YEAR"])
y = df["TARGET"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
# df.shape

In [ ]:
# df_test = df[df["YEAR"] == 2017]
# df_train = df[df["YEAR"] != 2017]
# X_train = df_train.drop(columns=["TARGET", "YEAR"])
# y_train = df_train["TARGET"]
# X_test = df_test.drop(columns=["TARGET", "YEAR"])
# y_test = df_test["TARGET"]

In [ ]:
#df = df[df["TARGET"].between(df["TARGET"].quantile(.05), df["TARGET"].quantile(.95))]

In [7]:
from sklearn.linear_model import LassoCV
reg = LassoCV()
reg.fit(X_train,y_train)
print(reg.alpha_)
print(reg.score(X_train,y_train))
coef = pd.Series(reg.coef_, index=X_train.columns)
print("Lasso picked " + str(sum(coef != 0)) + " variables and eliminated the other "+ str(sum(coef==0)) + " variables")
imp_coef = coef.sort_values(ascending=False)

1.2550689303189884
0.34505644755179876
Lasso picked 23 variables and eliminated the other 18 variables


In [8]:
feats = list(imp_coef[imp_coef > 0].index)
X_train = X_train[feats]
X_test = X_test[feats]

In [ ]:
feats

In [ ]:
# # Grid Search
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.model_selection import GridSearchCV

# regressor = RandomForestRegressor()
# parameters = [{'n_estimators' : [150,200,250,300], 'max_features' : ['auto','sqrt','log2']}]
# grid_search = GridSearchCV(estimator = regressor, param_grid = parameters,n_jobs=-1)
# grid_search = grid_search.fit(X_train, y_train)
# best_parameters = grid_search.best_params_

In [ ]:
best_parameters

In [9]:
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(n_estimators = 200, max_features = 'sqrt', max_depth=7,min_samples_leaf=2) 
regressor.fit(X_train,y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=7, max_features='sqrt', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=2,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=200, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [10]:
def mape(y_pred,y_true):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [11]:
from sklearn import metrics
y_pred = regressor.predict(X_test) 
y_pred_train = regressor.predict(X_train) 
print("Root Mean Squared Error Train: " + str(np.sqrt(metrics.mean_squared_error(y_pred_train,y_train))))
print("Root Mean Squared Error Test: " + str(np.sqrt(metrics.mean_squared_error(y_test,y_pred))))
print("Score Test: "+ str(regressor.score(X_test,y_test))) #R^2
print("MAPE Test:" + str(mape(y_pred,y_test)))

Root Mean Squared Error Train: 4.132060287063923
Root Mean Squared Error Test: 5.025289830844431
Score Test: 0.3632941306505074
MAPE Test:63.59078507769262


In [12]:
from sklearn import metrics
y_pred = X_test["PREVIOUS"]
y_pred_train = X_train["PREVIOUS"]
print("Root Mean Squared Error Train: " + str(np.sqrt(metrics.mean_squared_error(y_pred_train,y_train))))
print("Root Mean Squared Error Test: " + str(np.sqrt(metrics.mean_squared_error(y_test,y_pred))))
print("MAPE Test:" + str(mape(y_pred,y_test)))

Root Mean Squared Error Train: 5.916128843814246
Root Mean Squared Error Test: 5.864379826987876
MAPE Test:63.45772655863158
